# Capstone

Michael Schillawski, 10 April 2018

Data Science Immersive, Generaly Assembly

## Imports

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from pandas.io.json import json_normalize

%matplotlib inline

os.getcwd()

'/Users/mjschillawski/Google Drive/Data/generalassembly/12_capstone'

In [54]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/'
file = 'data_records_27638.txt'

data = pd.read_table(path+file,header=None,names=['recipe'],index_col=1)

In [55]:
data.head()

,recipe
1,Mushroom-risotto-352492
2,Filipino-bbq-pork-skewers-352163
3,Mushroom-and-Roasted-Garlic-Risotto-525026
4,Gratin-Dauphinois-_scalloped-Potatoes-With-Che...
5,Delicious-Grilled-Hamburgers-Allrecipes


In [56]:
data.tail()

,recipe
27634,Saltimbocca-alla-Romana-824004
27635,Carrotini-MyRecipes-241041
27636,Brioche-531830
27637,Mexican-Hot-Chocolate-Doughnuts-501510
27638,Thai-Curry-Burrito-Bowl-with-Sweet-and-Spicy-T...


In [57]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/metadata27638/'

recipes = []

for i in data.index:
    num = str(i)
    while len(num) < 5:
        num = '0' + num
        
    # https://stackoverflow.com/questions/28373282/how-to-read-a-json-dictionary-type-file-with-pandas
    with open(path+'meta'+num+'.json') as json_data:
        recipe = json.load(json_data)
        recipes.append(recipe)

recipes = json_normalize(recipes)   
recipes

,attributes.course,attributes.cuisine,attributes.holiday,attribution.html,attribution.logo,attribution.text,attribution.url,cookTime,cookTimeInSeconds,flavors.Bitter,...,nutritionEstimates,prepTime,prepTimeInSeconds,rating,source.sourceDisplayName,source.sourceRecipeUrl,source.sourceSiteUrl,totalTime,totalTimeInSeconds,yield
0,[Side Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom Risotto recipes: information powered ...,http://www.yummly.com/recipe/Mushroom-risotto-...,NaN,NaN,NaN,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2009/10/risotto-is-...,http://www.skinnytaste.com,30 minutes,1800.0,servings: 6
1,[Main Dishes],[Barbecue],NaN,<a href='http://www.yummly.com/recipe/Filipino...,http://static.yummly.com/api-logo.png,Filipino BBQ Pork Skewers recipes: information...,http://www.yummly.com/recipe/Filipino-bbq-pork...,NaN,NaN,0.8333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2008/08/filipino-bb...,http://www.skinnytaste.com,40 min,2400.0,None
2,[Main Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom and Roasted Garlic Risotto recipes: i...,http://www.yummly.com/recipe/Mushroom-and-Roas...,NaN,NaN,1.0000,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,3,MyRecipes,http://www.myrecipes.com/recipe/mushroom-roast...,http://www.myrecipes.com,1 Hr 25 Min,5100.0,Serves 6 (serving size: about 1 cup)
3,[Side Dishes],"[French, American]",NaN,<a href='http://www.yummly.com/recipe/Gratin-D...,http://static.yummly.com/api-logo.png,Gratin Dauphinois (Scalloped Potatoes with Che...,http://www.yummly.com/recipe/Gratin-Dauphinois...,NaN,NaN,0.6667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,4,MyRecipes,http://www.myrecipes.com/recipe/gratin-dauphin...,http://www.myrecipes.com,55 min,3300.0,7 servings (serving size: 1 cup)
4,[Main Dishes],[Barbecue],NaN,<a href='http://www.yummly.com/recipe/Deliciou...,http://static.yummly.com/api-logo.png,Delicious Grilled Hamburgers recipes: informat...,http://www.yummly.com/recipe/Delicious-Grilled...,10 Min,600.0,0.1667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",5 Min,300.0,4,AllRecipes,http://allrecipes.com/Recipe/delicious-grilled...,http://www.allrecipes.com,15 Min,900.0,3 servings
5,[Main Dishes],[American],"[Winter, Christmas, Thanksgiving]",<a href='http://www.yummly.com/recipe/Hickory_...,http://static.yummly.com/api-logo.png,Hickory-Smoked Bourbon Turkey recipes: informa...,http://www.yummly.com/recipe/Hickory_smoked-Bo...,NaN,NaN,0.8333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,4,MyRecipes,http://www.myrecipes.com/recipe/hickory-smoked...,http://www.myrecipes.com,None,NaN,Makes 12 to 14 servings
6,[Main Dishes],[American],NaN,<a href='http://www.yummly.com/recipe/Marinate...,http://static.yummly.com/api-logo.png,Marinated Beef Tenderloin recipes: information...,http://www.yummly.com/recipe/Marinated-Beef-Te...,NaN,NaN,0.6667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",25 Min,1500.0,3,Martha Stewart,http://www.marthastewart.com/962284/marinated-...,http://www.marthastewart.com,3 Hr 30 Min,12600.0,servings: 8
7,[Desserts],[Italian],[Summer],<a href='http://www.yummly.com/recipe/Cherry-T...,http://static.yummly.com/api-logo.png,Cherry Tortoni recipes: information powered by...,http://www.yummly.com/recipe/Cherry-Tortoni-My...,NaN,NaN,0.3333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,3,MyRecipes,http://www.myrecipes.com/recipe/cherry-tortoni...,http://www.myrecipes.com,None,NaN,10 servings (serving size: 1 [3-inch] tortoni ...
8,[Main Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Crock-Po...,http://static.yummly.com/api-logo.png,Crock-Pot Italian Turkey Meatballs and Homemad...,http://www.yummly.com/recipe/Crock-Pot-Italian...,NaN

In [60]:
short_recipes = recipes[['attributes.course','attributes.cuisine','name','ingredientLines']]

In [64]:
short_recipes['ingredientLines'][17464]

['9 ounces farro perlato (semi-pearled emmer), about 1 1/3 cups uncooked (see substitutes below)',
 '1 teaspoon chia seed or ground flaxseeds',
 '3 medium zucchini, trimmed and halved lengthwise',
 '1/2 red bell pepper, seeded',
 '12-16 ounces cherry tomatoes, halved',
 '2 green onions, green parts only, thinly sliced',
 '2 tablespoons white balsamic or white wine vinegar',
 '1 tablespoon lemon juice',
 '2 teaspoons spicy or Dijon mustard, or to taste',
 '1 large clove garlic, minced or pressed',
 '1/4 teaspoon salt (optional)',
 '1/8 teaspoon freshly ground black pepper',
 '1/4 cup chopped fresh basil',
 'Salt and black pepper, to taste',
 'Additional lemon juice, to taste']

## Natural Language Processing

In [ ]:
strip numbers
strip common measurements and their abbreviations
strip punctuation
strip preparation methods
strip 'to taste'

inventory my spice rack
strip spices / pantry items

## Recommendation Engine

In [ ]:
# figure out input format?
# maybe input prompt
# or url scrape?

In [ ]:
# inventory the differences
# suggest additional ingredients in order to complete both recipes